# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [2]:
# Install required dependencies
import sys
!{sys.executable} -m pip install "chromadb>=1.0.4" "openai>=1.73.0" "pydantic>=2.11.3" "python-dotenv>=1.1.0" "tavily-python>=0.5.4" "pdfplumber" "tavily-python"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [3]:
import os
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
import json
import chromadb
from tavily import TavilyClient

In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("BASE_URL")
tavily_api_key = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# It should use chroma client and collection you created
@tool
def retrieve_game(query: str) -> list:
  """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry.

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
  """
  chroma_client = chromadb.PersistentClient(path="chromadb")
  collection = chroma_client.get_collection("udaplay")
  result = collection.query(query_texts=[query], n_results=5)
  formated_result = []
  result_lenght = len(result["documents"][0])
  for i in range(result_lenght):
    formated_result.append(f"{i+1}. {result['documents'][0][i]}")
  return "\n".join(formated_result)


#### Evaluate Retrieval Tool

In [6]:
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
  """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
  """
  llm = LLM(model="gpt-3.5-turbo", openai_api_key=openai_api_key, base_url=base_url)
  system_message = SystemMessage(content="You are an expert AI assistant that helps people find information.")
  user_message = UserMessage(content=f"Your task is to evaluate if the documents are enough to respond the query.\n"
  f"Question: {question}\n"
  f"Documents: {retrieved_docs}\n"
  f"Give a detailed explanation, so it's possible to take an action to accept it or not.")

  # Use the LLM directly instead of Agent
  messages = [system_message, user_message]
  response = llm.invoke(messages)
  return response.content

#### Game Web Search Tool

In [7]:
@tool
def game_web_search(question: str) -> str:
  """
    Search the web using Tavily to find relevant information about the game industry.
    args:
    - question: a question about game industry.

    The result includes:
    - A summary of the most relevant information found on the web.
  """
  from tavily import TavilyClient

  tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
  search_results = tavily.search(query=question, max_results=5)

  if not search_results or 'results' not in search_results:
    return "No results found."

  results = search_results['results']
  formatted_results = []
  for i, result in enumerate(results):
    title = result.get('title', 'No Title')
    snippet = result.get('snippet', 'No Snippet')
    url = result.get('url', 'No URL')
    formatted_results.append(f"{i+1}. {title}\n{snippet}\n{url}\n")

  return "\n".join(formatted_results)

### Agent

In [8]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions
# Plug all Tools you developed

tools = [retrieve_game, evaluate_retrieval, game_web_search]

agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    openai_api_key=openai_api_key,
    base_url=base_url,
    instructions=(
        "You're UdaPlay, an AI Research Agent specializing in the video game industry. "
      "You can answer multistep questions by sequentially calling functions."
      "You follow a pattern of Thought and Action. "
      "Create a plan of execution: "
      "- Use Thought to describe your thoughts about the question you have been asked. "
      "- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly."
      "Your expertise includes game platforms, release dates, genres, and industry information. "
      "When answering questions about video games: "
      "1. First search the internal game database using retrieve_game "
      "2. Evaluate if the results are sufficient using evaluate_retrieval "
      "3. If needed, search the web for additional information using game_web_search "
      "When you think it's over, return the answer "
      "Never try to respond directly if the question needs a tool. "
      "But if you don't have a tool available, you can respond directly. "
        f"The actions you have are the Tools: {tools}. \n"
    )
)

In [9]:
def print_final_answer(question: str, final_state: dict):
  """
    Helper function to print the final answer from the agent's state.
    It looks for the last AIMessage in the messages list and prints its content.
  """
  if final_state and "messages" in final_state:
    for msg in reversed(final_state["messages"]):
      if isinstance(msg, AIMessage) and msg.content:
          print(f"Question: {question}")
          print(f"Final Answer: {msg.content}")
          break

In [10]:
questions = [
"When Pokémon Gold and Silver was released?",
"Which one was the first 3D platformer Mario game?",
"Was Mortal Kombat X realeased for Playstation 5?"
]

# Invoke your agent
for index, question in enumerate(questions):
  response = agent.invoke(query=question, session_id=index)
  final_state = response.get_final_state()
  print_final_answer(question, final_state)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Question: When Pokémon Gold and Silver was released?
Final Answer: Pokémon Gold and Silver were released in 1999 for the Game Boy Color.
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Question: Which one was the first 3D platformer Mario game?
Final Answer: The first 3D platformer Mario game is **Super Mario 64**, released in 1996 for the Nintendo 6

### (Optional) Advanced

In [11]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes